In [ ]:
# 修改run_4model函数签名，添加market_states参数
def run_4model(df, features_column, label_column, date_column, tic_column,
              unique_ticker, unique_datetime, trade_date, 
              first_trade_date_index=20,
              testing_windows=4,
              max_rolling_window_index=44,
              market_states=None):  # 新增参数
    # 初始化...
    
    for i in range(first_trade_date_index, len(unique_datetime)):
        try:
            # 获取当前市场状态
            current_market_state = None
            if market_states and unique_datetime[i] in market_states:
                current_market_state = market_states[unique_datetime[i]]
                print(f"Current market state: {current_market_state['state']} with confidence {current_market_state['confidence']}")
            
            # 准备训练数据...
            
            # 根据市场状态调整训练参数或选择不同的模型
            if current_market_state:
                if current_market_state['state'] == 'Bull':
                    # 牛市策略偏向进攻性，如增加小市值成长股权重
                    pass
                elif current_market_state['state'] == 'Bear':
                    # 熊市策略偏向防御性，如增加大市值价值股权重
                    pass
                elif current_market_state['state'] == 'Sideways':
                    # 震荡市策略偏向中性，如增加动量因子权重
                    pass
            
            # 训练模型...
            
            # 修改评估标准，根据市场状态调整
            if current_market_state:
                # 例如，在熊市中更重视风险控制
                if current_market_state['state'] == 'Bear':
                    # 为风险调整评估指标添加额外权重
                    rf_eval = evaluate_model(rf_model, X_test, y_test, risk_aversion=1.5)
                    xgb_eval = evaluate_model(xgb_model, X_test, y_test, risk_aversion=1.5)
                    gbm_eval = evaluate_model(gbm_model, X_test, y_test, risk_aversion=1.5)
                else:
                    rf_eval = evaluate_model(rf_model, X_test, y_test)
                    xgb_eval = evaluate_model(xgb_model, X_test, y_test)
                    gbm_eval = evaluate_model(gbm_model, X_test, y_test)
            else:
                # 原有评估代码...
            
            # 记录市场状态信息
            if current_market_state:
                df_best_model_name.loc[unique_datetime[i], 'market_state'] = current_market_state['state']
            
            # 剩余代码...
            
        except Exception:
            traceback.print_exc()
    
    # 修改返回值，添加市场状态信息
    df_market_states = pd.DataFrame(columns=['state', 'bull_confidence', 'bear_confidence', 'sideways_confidence'], index=trade_date)
    for date in trade_date:
        if market_states and date in market_states:
            state_info = market_states[date]
            df_market_states.loc[date, 'state'] = state_info['state']
            for state, conf in state_info['confidence'].items():
                df_market_states.loc[date, f'{state.lower()}_confidence'] = conf
    
    return (
            df_predict_rf, 
            df_predict_gbm,
            df_predict_xgb,
            df_predict_best,
            df_best_model_name, 
            evaluation_record,
            df_evaluation,
            df_market_states)  # 新增返回值

# 修改评估函数，添加市场状态相关的风险权重
def evaluate_model(model, X_test, y_test, risk_aversion=1.0):
    # 现有代码...
    
    # 根据风险厌恶度调整评估指标
    if risk_aversion != 1.0:
        # 增加对下行风险的惩罚
        y_predict = model.predict(X_test)
        downside_risk = np.mean(np.maximum(0, y_test - y_predict)**2)
        mse = mse + (risk_aversion - 1.0) * downside_risk
    
    return mse

# 修改保存函数
def save_model_result(sector_result, sector_name, market_states=None):
    # 现有代码...
    
    # 保存市场状态信息
    if market_states and len(sector_result) > 7:
        df_market_states = sector_result[7]
        df_market_states.to_csv(f'results/{sector_name}/df_market_states.csv')